# Урок 7. Тюнинг гиперпараметров, построение ансамблей алгоритмов.

**Задание 1 : Обучить алгоритмы LightGBM и XGBoost, получить OOF прогнозы, оценить корреляцию прогнозов на обучающей выборке. Применить модели на тестовую выборку и оценить корреляцию.**

Будет сделано позже